In [1]:
import pickle
import json
import geocoder
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
with open('flight_data_departure.html', 'r', encoding='utf-8') as f:
    departure = f.read()

In [3]:
dep_soup = BeautifulSoup(departure, 'html.parser')

In [4]:
dep_basis = {
    "departure":{
        "name":'羽田空港',
        "code":'HND',
        "lat":35.54572095,
        "lon":139.780587131238,
    },
    "arrival":{
        "name":None,
        "code":None,
        "lat":None,
        "lon":None,
    },
    "departure_time":None,
    "flight_number":None,
}

In [5]:
arr_basis = {
    "departure":{
        "name":None,
        "code":None,
        "lat":None,
        "lon":None,
    },
    "arrival":{
        "name":"羽田空港",
        "code":"HND",
        "lat":35.54572095,
        "lon":139.780587131238,
    },
    "departure_time":None,
    "flight_number":None,
}

In [6]:
dep_name = []
for td in dep_soup.find_all('tr'):
    td = td.find_all('td')
    airport_name = td[2].text
    dep_name.append(airport_name)

In [7]:
with open('flight_data_arrival.html', 'r', encoding='utf-8') as f:
    arrival = f.read()

In [8]:
arr_soup = BeautifulSoup(arrival, 'html.parser')

In [9]:
arr_name = []
for td in arr_soup.find_all('tr'):
    td = td.find_all('td')
    airport_name = td[2].text
    arr_name.append(airport_name)

In [10]:
airport_name = set(dep_name) | set(arr_name)

In [11]:
latlon = {}
changed_name = {}
for name in tqdm(airport_name):
    ret = geocoder.osm(name+'空港', timeout=5.0)
    if ret.latlng is None:
        print(name)
        true_name = input('enter airport name: ')
        changed_name[name] = true_name
        airport_name.remove(name)
        airport_name.add(true_name)
        ret = geocoder.osm(true_name+'空港', timeout=5.0)
        latlon[true_name] = ret.latlng
        continue
    latlon[name] = ret.latlng

 17%|█████████████▊                                                                     | 8/48 [00:08<00:43,  1.10s/it]

大阪（伊丹）
enter airport name: 大阪国際


 19%|███████████████▌                                                                   | 9/48 [00:14<01:37,  2.50s/it]

名古屋（中部）
enter airport name: 中部国際


 23%|██████████████████▊                                                               | 11/48 [00:20<01:34,  2.55s/it]

...

 94%|████████████████████████████████████████████████████████████████████████████▉     | 45/48 [01:09<00:05,  1.76s/it]

沖縄（那覇）
enter airport name: 那覇


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [01:16<00:00,  1.88s/it]

大阪（関西）
enter airport name: 関西国際


51it [01:24,  1.66s/it]                                                                                                


In [12]:
with open('data/latlon.pkl', 'wb') as f:
    pickle.dump(latlon, f)

In [ ]:
airport_code = {}
for name in airport_name:
    print('=======================')
    print(f'ariport: {name}')
    code = input('enter airport code: ')
    print(code)
    airport_code[name] = code

In [ ]:
with open('data/airport_code.pkl', 'wb') as f:
    pickle.dump(airport_code, f)

In [13]:
with open('airport_code.pickle', 'rb') as f:
    airport_code = pickle.load(f)

In [17]:
dep_data = []
deparr = 'arrival'
for td in tqdm(dep_soup.find_all('tr')):
    td = td.find_all('td')
    info = dep_basis.copy()
    airport_name = td[2].text
    if airport_name in changed_name.keys():
        airport_name = changed_name[airport_name]
    info[deparr]['name'] = airport_name+"空港"
    info[deparr]['code'] = airport_code[airport_name]
    lat, lon = latlon[airport_name]
    info[deparr]['lat'] = lat
    info[deparr]['lon'] = lon
    info['departure_time'] = td[0].text[2:]
    info['flight_number'] = td[4].text
    dep_data.append(info)

100%|█████████████████████████████████████████████████████████████████████████████| 579/579 [00:00<00:00, 32194.16it/s]


In [19]:
arr_data = []
deparr = 'departure'
for td in tqdm(arr_soup.find_all('tr')):
    td = td.find_all('td')
    info = arr_basis.copy()
    airport_name = td[2].text
    if airport_name in changed_name.keys():
        airport_name = changed_name[airport_name]
    info[deparr]['name'] = airport_name+"空港"
    info[deparr]['code'] = airport_code[airport_name]
    lat, lon = latlon[airport_name]
    info[deparr]['lat'] = lat
    info[deparr]['lon'] = lon
    info['departure_time'] = td[0].text[2:]
    info['flight_number'] = td[4].text
    arr_data.append(info)

100%|█████████████████████████████████████████████████████████████████████████████| 580/580 [00:00<00:00, 36343.07it/s]


In [21]:
with open('data/flight_info_departure.pkl', 'wb') as f:
    pickle.dump(dep_data, f)

In [22]:
with open('data/flight_info_arrival.pkl', 'wb') as f:
    pickle.dump(arr_data, f)

In [24]:
ret = geocoder.osm('羽田空港', timeout=5.0)
latlon['羽田'] = ret.latlng

In [25]:
code_latlon = {}
for name, code in airport_code.items():
    code_latlon[code] = latlon[name]

In [27]:
with open('data/airportCode2latlon.pkl', 'wb') as f:
    pickle.dump(code_latlon, f)